# **Readme:
SAM_geo.ipynb   --基于SAM模型的遥感卫星影像分割
*   功能:
1. 完成云端或者本地任意范围、尺度、分辨率卫星影像的加载、交互和下载。
2. 使用SAM分割模型完成影像分割任务,并将分割结果对象保存为带有地理参考的.tif和.shp文件
3. 在云端实时查看分割结果


*   使用方法:

为了加快运行的效率,减小工具包和SAM模型安装的时间,建议在Google Colab平台上使用GPU模式运行



*   参考文献:
1. Kirillov, A., Mintun, E., Ravi, N., Mao, H., Rolland, C., Gustafson, L., ... & Girshick, R. (2023). Segment anything. arXiv preprint arXiv:2304.02643.https://doi.org/10.48550/arXiv.2304.02643
2. Qiusheng Wu. samgeo: A Python package for segmenting geospatial data with the Segment Anything Model (SAM)


* 参考代码:

  https://github.com/opengeos/segment-geospatial/tree/main











# 1.安装并导入工具包
工具包内容:
segment-geospatial
leafmap
localtileserver

取消下句代码的注释,在GPU运行类型下即可完成必需工具包的安装
安装可能需要1-2分钟,请耐心等待
完成安装后,直接导入即可

In [ ]:
# %pip install segment-geospatial leafmap localtileserver

In [ ]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

# 2.创建交互地图

In [ ]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19) # 设置地图中心的经纬坐标和聚焦等级
m.add_basemap("SATELLITE")  # 将底图设置为卫星影像，可使用地图右上角的ToolBar更改底图类型
m  # 显示地图

**完成交互地图搭建后,使用左侧绘图工具手动绘制特定ROI区域,进行后续分割工作**


如果用户没有绘制ROI区域,会以[-95.3704, 29.6762, -95.368, 29.6775]区域为默认的分割对象

In [ ]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3714, 29.6746, -95.3673, 29.6779]

In [ ]:
m.user_roi_bounds()  # 查看划定的ROI范围

# 3.下载地图
将上步ROI区域内的卫星地图下载为.tif格式,并自动为地图分配投影坐标


In [ ]:
image = "ROImage.tif"  # 地图名,可手动修改

In [ ]:
# image = 'path/image.tif'  # 也可以使用本地遥感图像

使用软件包中tms_to_geotiff()函数将卫星影像输出为.tif格式

参数设置:
1.   列表项
2.   列表项

结果会保存至云盘中,也可以导出到本地,在ArcMap、Envi、QGIS软件进行查看和操作

In [ ]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

查看下载的影像

In [ ]:
m.layers[-1].visible = False  # 关闭卫星底图,从而更直观地查看影像位置
m.add_raster(image, layer_name="Image")
m

# 4.SAM分割
完成SAM模型的搭建、影像分割、结果查看并保存为.shp矢量格式和.tif栅格灰度图格式,便于在ArcGIS等软件下执行操作

① SAM模型初始化

默认下载并使用SAM中的"vit_h"模型,执行分割任务

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

② 影像分割

建议在GPU环境下运行,加快运行效率,运行时间根据图像的大小会有差别

参数设置中:
1. 将batch参数设置为True,即对原图像进行分块分割,防止输入图像过大造成运行效率降低
2. 将erosion_kernel侵蚀单元设为3×3,mask灰度图的值域设为0-255

In [ ]:
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

③ 将分割结果保持为.shp矢量文件

In [ ]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

④ 云端查看分割结果

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(shapefile, layer_name="Vector", style=style)
m